In [15]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [16]:
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import matplotlib.pyplot as plt
import scipy.stats as sci
import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
numpy 1.19.1
pandas 1.1.1
matplotlib 3.3.1


In [17]:
# mapping human gene symbol to entrez
dat=pd.read_table('data_ori/Sym2Entrez.txt',sep='\t')
dat.columns=['sym','entrez']
dat=dat.loc[dat['entrez'].notnull()]
dat=dat.loc[dat['sym'].notnull()]
dat['entrez']=dat['entrez'].astype(int)

e2s=dat.set_index('entrez')
e2s_dic=e2s['sym'].to_dict() # e2s_dic={1: 'A1BG', 503538: 'A1BG-AS1',...}

In [18]:
def merge_two_gene2(row):
    g1=row['g1']
    g2=row['g2']
    
    if g1<g2:
        return g1+'_'+g2
    else:
        return g2+'_'+g1
    
def merge_two_gene1(row):
    g1=row['mut_gene']
    g2=row['dep_gene']

    if g1<g2:
        return g1+'_'+g2
    else:
        return g2+'_'+g1

def get_GS():
    SDB=pd.read_table('data_ori/gold_standard_Set/SL_human (synlethalDB).txt',usecols=[1,3,8],header=None,sep='\t')
    SDB.columns=['g1','g2','score']

    SDB['g1']=SDB['g1'].map(e2s_dic)
    SDB['g2']=SDB['g2'].map(e2s_dic)

    SDB=SDB.loc[(SDB['g1'].notnull())&(SDB['g2'].notnull())]
    SDB

    SDB['mg']=SDB.apply(merge_two_gene2, axis=1)
    SDB=SDB.sort_values(by='score', ascending=False)
    SDB=SDB.drop(columns=['g1','g2','score'])
    GS=SDB.copy()
    return GS

def get_ALL(tp,W):
    a2=pd.read_table('result_SL/{}_M003_I10_{}_Exp_fdr.txt'.format(tp,W), sep='\t', usecols=[0,1,2,4])
    a2['mg']=a2.apply(merge_two_gene1, axis=1)
    a2=a2.drop(columns=['mut_gene','dep_gene'])
    return a2

def f1(r1):
    d1=r1['DST=1'][1:-1].replace("'","").split(', ')
    d1=[ii for ii in d1 if ii!='']
    d2=r1['DST=2'][1:-1].replace("'","").split(', ')
    d2=[ii for ii in d2 if ii!='']
    it=r1['INIT.'][1:-1].replace("'","").split(', ')
    it=[ii for ii in it if ii!='']
    
    d2m=d1+d2
    itm=d1+d2+it
    
    return pd.Series([itm, d2m, d1],index=['it','d2','d1'])

def get_SP(file_name):
    crp_KEGG=pd.read_table(file_name,sep='\t',index_col=0)
    crp_KEGG_merged=crp_KEGG.apply(f1, axis=1).reset_index().rename(columns={'index':'MUT'})
    crp_KEGG_melt=crp_KEGG_merged.melt(id_vars='MUT', var_name='RP', value_name='SP')

    crp_KEGG_m2=pd.DataFrame(columns=['RP','MUT','SP','MUT_SP'])

    ind=0
    for idx in crp_KEGG_melt.index:
        row=crp_KEGG_melt.loc[idx]
        for sp in row['SP']:
            if row['MUT']<sp:
                crp_KEGG_m2.loc[ind]=[row['RP'],row['MUT'],sp, row['MUT']+'_'+sp]
            else:
                crp_KEGG_m2.loc[ind]=[row['RP'],row['MUT'],sp, sp+'_'+row['MUT']]
            ind+=1
            
    crp_KEGG_m2=crp_KEGG_m2.drop(columns=['MUT','SP'])        
    return crp_KEGG_m2

In [19]:
def eval_hypergeo(HT,AL,GS,SP):
    hypergeo_pval=sci.hypergeom.sf(len(HT),len(AL),len(GS),len(SP))
    return [len(HT),len(AL),len(GS),len(SP), hypergeo_pval]

In [20]:
def check_tval_pos(mg, AL_df_subset):
    if AL_df_subset.loc[AL_df_subset['mg']==mg, 't_val'].iloc[0]>0:
        return True
    else:
        return False
    
def func_check_tval_pos(SP_df):
    AL_df_subset=AL_df.loc[AL_df['mg'].isin(set(SP_df['MUT_SP']))]
    SP_pos_df=pd.DataFrame(columns=['RP','MUT_SP'])
    for ind, row in SP_df.iterrows():
        if check_tval_pos(row['MUT_SP'], AL_df_subset):
            SP_pos_df.loc[ind]=list(row)
    SP_pos_df.reset_index(drop=True, inplace=True)
    return SP_pos_df

In [21]:
GS_init=set(get_GS()['mg'])
for TP in ['CRISPER','shRNA']:
    for W in ['WO','WI']:
        print('#######', TP, '##', W)
        a2=pd.read_table('result_SL/{}_M003_I10_{}_Exp_fdr.txt'.format(TP,W), sep='\t', usecols=[0,1,2,4])
        print(len(a2))
        SP=a2.loc[(a2['fdr']<0.2)&(a2['t_val']>0)]
        print(len(SP))
        SP=a2.loc[(a2['fdr']<0.2)&(a2['t_val']<=0)]
        print(len(SP))
        SP=a2.loc[(a2['fdr']<0.2)]
        print(len(SP))


####### CRISPER ## WO
81008150
389
1234
1623
####### CRISPER ## WI
72278365
418
1322
1740
####### shRNA ## WO
34019928
519
940
1459
####### shRNA ## WI
31734111
480
909
1389


In [ ]:
tp2tp={'CRISPER':'crp','shRNA':'shr'}

GS_init=set(get_GS()['mg'])

res_df=pd.DataFrame(columns=['screening','zero_exp','NET','RP','HIT','ALL','GS','SP','hyp_pval'])
## WO ==> including zero expression
## WI ==> excluding zero expression
ind=0

HT_dic={}

for TP in ['CRISPER','shRNA']:
    for W in ['WO','WI']:
        AL_df=get_ALL(TP, W)
        AL=set(AL_df['mg'])
        GS=GS_init & AL
        SP=set(AL_df.loc[(AL_df['fdr']<0.2)&(AL_df['t_val']>0), 'mg'])
        HT=GS&SP
        
        HT_dic[(TP,W, 0,'HT')]=HT
        HT_dic[(TP,W, 0,'SP')]=SP
        
        res=eval_hypergeo(HT,AL,GS,SP)
        res_df.loc[ind]=[TP,{'WO':'IZ','WI':'EZ'}[W],np.nan,np.nan]+res
        ind+=1

    for NET in ['KEGG','PPI']:
        print('########', TP, NET)
        SP_df=get_SP('result_main/{}_diff_table_{}.txt'.format(tp2tp[TP], NET))
        print(SP_df)
        SP_pos_df=func_check_tval_pos(SP_df)
        print(SP_pos_df)
        
        for RP in ['it','d2','d1']:
            SP=set(SP_pos_df.loc[SP_pos_df['RP']==RP,'MUT_SP'])
            HT=SP&GS
            
            HT_dic[(TP,NET, RP,'HT')]=HT
            HT_dic[(TP,NET, RP,'SP')]=SP
            
            res=eval_hypergeo(HT,AL,GS,SP)
            res_df.loc[ind]=[TP,{'WO':'IZ','WI':'EZ'}[W],NET,RP]+res
            ind+=1

In [ ]:
res_df.to_csv('eval_result/synlethDB_results.txt', sep='\t')